In [1]:
# !pip install opencv-python
# !pip install numpy
# !pip install mediapipe opencv-python
# !pip install pandas
# !pip install xmltodict
# !pip install lxml

In [2]:
import cv2
import mediapipe as mp
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

#mediapipe
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

# Yolo 로드
net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")
classes = []
with open("coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]
layer_names = net.getLayerNames()
output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]

try: #name 폴더 만들기
    os.makedirs("./preprocessing")
except:
    pass

## XML 정보 불러오기

In [3]:
from lxml import etree
from PIL import Image
import xmltodict, cv2, os
import xml.etree.ElementTree as ET

In [4]:
#xml 파일 불러옴
def load_xmldata(save_path, video_name):
    xml_file = '{0}.xml'.format(video_name)
    doc = ET.parse(xml_file)
    root = doc.getroot()
    frame_list = []
    action_list = []
    action_name = ['normal']
    try:
        os.makedirs("{0}/{1}".format(save_path, 'normal'))
    except:
        pass
    
    action_list=root.findall("./object/action")
    for i in action_list:
        for frame in i.findall("frame"):
            frame_list.append([i.find("actionname").text, int(frame.find("start").text), int(frame.find("end").text)])
    for action in root.iter("action"):
        action_name.append(action.find("actionname").text)
        try: 
#            print("{0}/{1}".format(root_dir, i[:-4]))
            os.makedirs("{0}/{1}".format(save_path, action.find("actionname").text))

        except:
            pass
        
    return frame_list

## 영상 프레임 자르기

In [5]:
def frame_cutting(save_path, video_dir, frame_list):
    vidcap = cv2.VideoCapture(video_dir)
    count = 0
    success = True
#     print('cutting')
    while success:
        try:
            success, image = vidcap.read()
            if(int(vidcap.get(1)) % 1 == 0):
#                 print('Saved frame number : ' + str(int(vidcap.get(1))))
                count += 1
                state = 0
                temp_row = []
#                 row = [count, 'none' , 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
#                 print("{0}/{1}/{2}.jpg".format(save_path, 'normal', count))
                for i in frame_list:
                    if i[1]<= count and count <= i[2]:
                        image = cv2.resize(image, (1152,648))
                        image, temp_row = pose_detect(image, count, i[0])
                        if 20 < len(temp_row):
#                             row = temp_row
                            w_csv(save_path, ntpath.basename(video_dir), temp_row)
                            cv2.imwrite("{0}/{1}/{2}.jpg".format(save_path, i[0], count), image)
                        
#                         print("{0}/{1}/{2}.jpg".format(save_path, i[0], count))
                        state = 1
                        break
#                     print('Saved frame%d.jpg' % count)
                if state == 1: #이미 action에서 분석된 이미지면 normal에서는 분석하지 않고 넘어감
                    continue
            
                if count % 60 == 0:
                    image = cv2.resize(image, (1152,648))
                    image, temp_row = pose_detect(image, count, 'normal')
                    if 20 < len(temp_row):
#                         row = temp_row
                        w_csv(save_path, ntpath.basename(video_dir), temp_row)
                        cv2.imwrite("{0}/{1}/{2}.jpg".format(save_path, 'normal', count), image) #사진 저장

        except:
            pass
    vidcap.release()

## 폴더 생성

In [6]:
#name 폴더 만들기
def make_dir(root_dir, dir_name):
    folder_list = []
    for i in dir_name:
        try: 
            if i not in folder_list:
#                 print("{0}/{1}".format(root_dir, i[:-4]))
                os.makedirs("{0}/{1}".format(root_dir, i[:-4]))
                folder_list.append(dir_name)

        except:
            pass
    return folder_list

## 파일 검색

In [7]:
import pathlib
import os
import ntpath

def find_file_fullname(fileDir):
    file_name = []
    for name in os.listdir(fileDir):
        file_name.append(fileDir + "/" + name)
    return file_name

def find_mp4(fileDir, fileExt):
    video_name = []
    for name in pathlib.Path(fileDir).glob(fileExt):
        video_name.append(str(name))
    return video_name

def path_to_filename(fileDir):
    file_name = []
    for name in fileDir:
        file_name.append(ntpath.basename(name))
    return file_name

## csv 쓰기

In [8]:
import csv
def w_csv(save_path, video_name, row):
    f = open('{0}/{1}_landmark.csv'.format(save_path, video_name),'a', newline='')
    wr = csv.writer(f)
    wr.writerow(row)
    f.close()
#     print('write csv')

#     f = open('landmark.csv','a', newline='')
#     wr = csv.writer(f)
#     wr.writerow([3,'최번개', '강원'])
#     f.close()

## 영상 처리

In [9]:
# img = cv2.imread("C:/Users/BVer/are_you_ok/img/pose10.jpg")
#img = cv2.resize(img, None, fx=0.4, fy=0.4)

def pose_detect(img, name, action):
#     print('detect start')
    row = []
    
    height, width, channels = img.shape
    img.flags.writeable = True
    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

    
    #human detect
    blob = cv2.dnn.blobFromImage(img, 0.00392, (512, 512), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outs = net.forward(output_layers)

    class_ids = []
    confidences = []
    boxes = []
    for out in outs: #yolo object detect
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5  and class_id == 0:
                # Object detected
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)
                # 좌표
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)
                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

    #pose detect on human object
    index = 0
    new_img = img
    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]
            #frame = img[y:y+h, x:x+w]
            frame = img[y-3:y+h+6, x-3:x+w+6]

            #mediapipe pose detect on yolo object frame
            with mp_pose.Pose(model_complexity=2, min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
                # Recolor image to RGB
                smallimg = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                smallimg.flags.writeable = False

                # Make detection
                results = pose.process(smallimg)

                # Render detections
                #mp_drawing.draw_landmarks(smallimg, results.pose_landmarks, mp_pose.POSE_CONNECTIONS) #조각별로 보기
                row += processing_landmark(results.pose_landmarks, x, y, w, h, width, height, name, action)
                mp_drawing.draw_landmarks(new_img, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
                index += 1
#                     plt.imshow(smallimg)
#                     plt.show()
#     plt.imshow(new_img)
#     plt.show()

        #Recolor back to BGR plt로 출력할땐 주석처리
    new_img.flags.writeable = True
    new_img = cv2.cvtColor(new_img, cv2.COLOR_RGB2BGR)
    return new_img, row

#     cv2.imwrite('save_path/{0}.jpg'.format(img), new_img)

## 좌표 벡터화

In [ ]:
if result.pose_landmarks is not None:
    for res in result.pose_landmarks:
        joint = np.zeros((33, 4))
        for j, lm in enumerate(res.landmark):
            joint[j] = [lm.x, lm.y, lm.z, lm.visibility]

        # Compute angles between joints
        v1 = joint[[11, 23, 15, 13, 11, 23, 25, 16, 14, 12, 24, 26], :3] # Parent joint
        v2 = joint[[12, 24, 13, 11, 23, 25, 27, 14, 12, 24, 26, 28], :3] # Child joint
        v = v2 - v1 # [20, 3]
        # Normalize v
        v = v / np.linalg.norm(v, axis=1)[:, np.newaxis]

        # Get angle using arcos of dot product
        angle = np.arccos(np.einsum('nt,nt->n',
            v[[0, 0, 0, 0, 1, 1, 1, 1, 2, 3, 4, 5, 7, 8, 9, 10],:], 
            v[[3, 4, 8, 9, 4, 5, 9, 10, 3, 4, 5, 6, 8, 9, 10, 11],:])) # [15,]

        angle = np.degrees(angle) # Convert radian to degree

        angle_label = np.array([angle], dtype=np.float32)
        angle_label = np.append(angle_label, idx)

        d = np.concatenate([joint.flatten(), angle_label])

        data.append(d)

        mp_drawing.draw_landmarks(img, res, mp_hands.HAND_CONNECTIONS)
        
data = np.array(data)
print(action, data.shape)
np.save(os.path.join('dataset', f'raw_{action}_{created_time}'), data)

# Create sequence data
full_seq_data = []
for seq in range(len(data) - seq_length):
    full_seq_data.append(data[seq:seq + seq_length])

full_seq_data = np.array(full_seq_data)
print(action, full_seq_data.shape)
np.save(os.path.join('dataset', f'seq_{action}_{created_time}'), full_seq_data)

## 랜드마크 위치 조절 및 리스트화

In [10]:
def processing_landmark(landmarks, x, y, w, h, ori_w, ori_h, name, index): 
    row_list = [name, index] #[프레임명, index ,머리 x, y, 어께 좌 x, y, 어께 우 x, y, 팔꿈치 좌 x, y, 팔꿈치 우 x, y, 골반 좌 x, y, 골반 우 x, y, 무릎 좌 x, y, 무릎 우 x, y, 발 좌 x, y, 발 우 x, y]
    head_x = []
    l_hand_x = []
    r_hand_x = []
    l_foot_x = []
    r_foot_x = []
    head_y = []
    l_hand_y = []
    r_hand_y = []
    l_foot_y = []
    r_foot_y = []
    temp = []
    error_count = 0
    for lndmrk in mp_pose.PoseLandmark:
        if lndmrk == 11:
            temp = sum_in_xy(head_x, head_y)
            row_list.append(temp[0])
            row_list.append(temp[1])
            
        elif lndmrk == 23:
            temp = sum_in_xy(l_hand_x, l_hand_y)
            row_list.append(temp[0])
            row_list.append(temp[1])
            
            temp = sum_in_xy(r_hand_x, r_hand_y)
            row_list.append(temp[0])
            row_list.append(temp[1])
    
        try:
            if landmarks.landmark[lndmrk.value].visibility >= 0.5: #min_detection_confidence
                landmarks.landmark[lndmrk.value].x = ((x - 3) + (w + 6) * landmarks.landmark[lndmrk.value].x)/ori_w
                landmarks.landmark[lndmrk.value].y = ((y - 3) + (h + 6) * landmarks.landmark[lndmrk.value].y)/ori_h
                if lndmrk < 11:
                    head_x.append(landmarks.landmark[lndmrk.value].x)
                    head_y.append(landmarks.landmark[lndmrk.value].y)
                elif 14 < lndmrk and lndmrk < 23:
                    if lndmrk % 2 == 0:
                        r_hand_x.append(landmarks.landmark[lndmrk.value].x)
                        r_hand_y.append(landmarks.landmark[lndmrk.value].y)
                    else:
                        l_hand_x.append(landmarks.landmark[lndmrk.value].x)
                        l_hand_y.append(landmarks.landmark[lndmrk.value].y)
                elif 26 < lndmrk:
                    if lndmrk % 2 == 0:
                        r_foot_x.append(landmarks.landmark[lndmrk.value].x)
                        r_foot_y.append(landmarks.landmark[lndmrk.value].y)
                    else:
                        l_foot_x.append(landmarks.landmark[lndmrk.value].x)
                        l_foot_y.append(landmarks.landmark[lndmrk.value].y)
                else:
                    row_list.append(landmarks.landmark[lndmrk.value].x)
                    row_list.append(landmarks.landmark[lndmrk.value].y)
            else:
                error_count += 1
                if lndmrk in [11, 12, 13, 14, 23, 24, 25, 26]:
                    row_list.append(0)
                    row_list.append(0)
        except:
            pass
        
    temp = sum_in_xy(l_foot_x, l_foot_y)
    row_list.append(temp[0])
    row_list.append(temp[1])

    temp = sum_in_xy(r_foot_x, r_foot_y)
    row_list.append(temp[0])
    row_list.append(temp[1])
    
    if 20 > len(row_list):
        row_list = []
        
    return(row_list)

In [11]:
def sum_in_xy(x_list, y_list):
    xy = [0, 0]
    if len(x_list) == 0:
        return xy
    else:
        sum = 0
        for i in range(0, len(x_list)):
            sum += x_list[i]
        xy[0] = sum
        sum = 0
        for i in range(0, len(y_list)):
            sum += y_list[i]
        xy[1] = sum
    return xy
    

# 실행

In [12]:
now = '/insidedoor_06'
root_dir = 'C:/Users/BVer/are_you_ok'
video_dir = root_dir + '/video/assult' + now

try: #name 폴더 만들기
    os.makedirs("./preprocessing" + now)
except:
    pass

sub_folder_name = []
sub_folder_name = find_file_fullname(video_dir)
# print(path_to_filename(sub_folder_name))
make_dir("./preprocessing{0}".format(now), path_to_filename(sub_folder_name))

i = []
print('preprocessing start')
for name in sub_folder_name:
    video_path = []
    video_path = find_mp4(name, r'*.mp4')
#     print(video_path)
    i = make_dir("./preprocessing{0}/{1}".format(now, ntpath.basename(name)), path_to_filename(video_path))
#     print(name)
#     print(path_to_filename(video_path))

    for file in video_path:
        save_path = "./preprocessing{0}/{1}/{2}".format(now, ntpath.basename(name), ntpath.basename(file)[:-4])
        print(ntpath.basename(file)[:-4])
        frame_cutting(save_path, file, load_xmldata(save_path, file[:-4]))
        print('-F----' + file + '-----')
    print('-F----' + name + '-----')
print('-F------ ' + now + ' -------')

preprocessing start
407-6_cam01_assault01_place04_day_spring
-F----C:\Users\BVer\are_you_ok\video\assult\insidedoor_06\407-6\407-6_cam01_assault01_place04_day_spring.mp4-----
407-6_cam01_assault01_place04_day_summer
-F----C:\Users\BVer\are_you_ok\video\assult\insidedoor_06\407-6\407-6_cam01_assault01_place04_day_summer.mp4-----
407-6_cam01_assault01_place04_day_winter
-F----C:\Users\BVer\are_you_ok\video\assult\insidedoor_06\407-6\407-6_cam01_assault01_place04_day_winter.mp4-----
407-6_cam02_assault01_place04_day_spring
-F----C:\Users\BVer\are_you_ok\video\assult\insidedoor_06\407-6\407-6_cam02_assault01_place04_day_spring.mp4-----
407-6_cam02_assault01_place04_day_summer
-F----C:\Users\BVer\are_you_ok\video\assult\insidedoor_06\407-6\407-6_cam02_assault01_place04_day_summer.mp4-----
407-6_cam02_assault01_place04_day_winter
-F----C:\Users\BVer\are_you_ok\video\assult\insidedoor_06\407-6\407-6_cam02_assault01_place04_day_winter.mp4-----
407-6_cam03_assault01_place04_day_spring
-F----C:\